# 01 - Find Files

**Search for the Files**
_______
Formatting the Directory-Pattern Dictionary
The function glob_multiple_file_paths expects a dictionary where each key-value pair corresponds to a root directory and a file pattern to search within that directory. The keys are the root directories where you want to start the search, and the values are the file patterns to match against.

Example Dictionary Format:

>dir_pattern_dict = {
>    '/path/to/first/root_dir': '*.nii',
>
>    '/path/to/second/root_dir': '*.nii.gz',
>
>    '/another/path': '*_label.nii'
>     Add more key-value pairs as needed
>}

Using Wildcards:

The file patterns can include wildcards to match multiple files:
- *: Matches zero or more characters
- **: Searches all directories recursively
- *.nii will match all files ending with .nii
- ?: Matches any single character
- file?.nii will match file1.nii, file2.nii, etc.
- [seq]: Matches any character in seq
- file[1-3].nii will match file1.nii, file2.nii, file3.nii
- [!seq]: Matches any character NOT in seq
- file[!1-3].nii will match any file that doesn't have 1, 2, or 3 in that position, like file4.nii, file5.nii, etc.

Feel free to combine these wildcards to create complex file patterns. For example, *_??.nii will match files like file_01.nii, file_02.nii, etc.

Where to Save to

In [7]:
# Define the dictionary with root directories and file patterns
dir_pattern_dict = {
    '/Volumes/HowExp/datasets/02a_Corbetta_Stroke_Lesion/BIDS_Dataset/sub-*/roi/': 'sub-*.nii.gz'}

## Glob the files and check to see if acceptable

In [8]:
save_files = False

In [9]:
from calvin_utils.file_utils.file_path_collector import glob_multiple_file_paths
import os
# Validate Directory
# os.mkdir(os.path.dirname(csv_path))
# Call the function and save the returned DataFrame to a CSV file
path_df = glob_multiple_file_paths(dir_pattern_dict, save=save_files, save_path=None)

# Display the saved path and the DataFrame
display(path_df)

,paths
0,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
1,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
2,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
3,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
4,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
...,...
130,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
131,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
132,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
133,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...


# Option 2 - Extract Subject IDs

In [10]:
# Define the preceding and proceeding strings
preceding = 'sub-'
proceeding = '/'

# Extract the substring and add it to a new column 'subject'
path_df['subject'] = path_df['paths'].str.extract(f'{preceding}(.*?){proceeding}')

# Display the updated DataFrame
display(path_df)

,paths,subject
0,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,026
1,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,088
2,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,043
3,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,044
4,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,072
...,...,...
130,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,038
131,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,053
132,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,098
133,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,062


In [11]:
path_df['subject'] = path_df['subject'].astype('int')

In [12]:
path_df

,paths,subject
0,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,26
1,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,88
2,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,43
3,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,44
4,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,72
...,...,...
130,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,38
131,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,53
132,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,98
133,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,62


# 03 Option A - Import Another CSV and Add the Paths to It
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [18]:
spreadsheet_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/metadata/mergedNotCerebellumExtensive.csv'
sheet = None #If using Excel, enter a string here

In [19]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=spreadsheet_path, output_dir=os.path.dirname(spreadsheet_path), sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,study_id,subject,MotorL_acute,MotorR_acute,Motor_IC_acute,MotorL_3month,MotorR_3month,Motor_IC_3month,MotorL_1year,MotorR_1year,...,gdss_11_1year,gdss_12_1year,gdss_13_1year,gdss_14_1year,gdss_15_1year,gdss_score_1year,clock_acute,mes_tot_miss_acute,file,conn_path
0,FCS024,24,NaN,NaN,NaN,-1.3652,0.5878,-0.7725,-1.0227,0.2821,...,0.0,1.0,0.0,0.0,0.0,3.0,6.0,45.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
1,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
2,FCS027,27,NaN,NaN,NaN,0.6425,-0.5805,0.0808,0.7433,0.3375,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
3,FCS029,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
4,FCS030,30,NaN,NaN,NaN,0.3583,0.5973,0.7771,-0.1659,0.6261,...,0.0,1.0,1.0,0.0,1.0,13.0,12.0,31.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,FCS192,192,0.7975,0.5861,0.8521,0.7416,0.4959,0.9150,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
131,NaN,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
132,NaN,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
133,FCS195,195,-2.3491,0.0868,-1.7040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,38.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...


In [20]:
merged_df = data_df.merge(path_df, on='subject', how='outer', suffixes=('', '_pathdf'))
display(merged_df)

,study_id,subject,MotorL_acute,MotorR_acute,Motor_IC_acute,MotorL_3month,MotorR_3month,Motor_IC_3month,MotorL_1year,MotorR_1year,...,gdss_12_1year,gdss_13_1year,gdss_14_1year,gdss_15_1year,gdss_score_1year,clock_acute,mes_tot_miss_acute,file,conn_path,paths
0,FCS024,24,NaN,NaN,NaN,-1.3652,0.5878,-0.7725,-1.0227,0.2821,...,1.0,0.0,0.0,0.0,3.0,6.0,45.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
1,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
2,FCS027,27,NaN,NaN,NaN,0.6425,-0.5805,0.0808,0.7433,0.3375,...,0.0,0.0,0.0,0.0,0.0,12.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
3,FCS029,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
4,FCS030,30,NaN,NaN,NaN,0.3583,0.5973,0.7771,-0.1659,0.6261,...,1.0,1.0,0.0,1.0,13.0,12.0,31.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,FCS192,192,0.7975,0.5861,0.8521,0.7416,0.4959,0.9150,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
131,NaN,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
132,NaN,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
133,FCS195,195,-2.3491,0.0868,-1.7040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,38.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...


Rename Paths Column

In [21]:
newname = 'roi_paths'
merged_df = merged_df.rename(columns={'paths': newname})
display(merged_df)

,study_id,subject,MotorL_acute,MotorR_acute,Motor_IC_acute,MotorL_3month,MotorR_3month,Motor_IC_3month,MotorL_1year,MotorR_1year,...,gdss_12_1year,gdss_13_1year,gdss_14_1year,gdss_15_1year,gdss_score_1year,clock_acute,mes_tot_miss_acute,file,conn_path,roi_paths
0,FCS024,24,NaN,NaN,NaN,-1.3652,0.5878,-0.7725,-1.0227,0.2821,...,1.0,0.0,0.0,0.0,3.0,6.0,45.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
1,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
2,FCS027,27,NaN,NaN,NaN,0.6425,-0.5805,0.0808,0.7433,0.3375,...,0.0,0.0,0.0,0.0,0.0,12.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
3,FCS029,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,13.0,2.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
4,FCS030,30,NaN,NaN,NaN,0.3583,0.5973,0.7771,-0.1659,0.6261,...,1.0,1.0,0.0,1.0,13.0,12.0,31.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,FCS192,192,0.7975,0.5861,0.8521,0.7416,0.4959,0.9150,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
131,NaN,193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
132,NaN,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...
133,FCS195,195,-2.3491,0.0868,-1.7040,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,38.0,/data/nimlab/dl_archive/Corbetta_Controls/sub-...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...,/Volumes/HowExp/datasets/02a_Corbetta_Stroke_L...


Save the merged df

In [22]:
merged_df.to_csv(spreadsheet_path, index=False)

# 03 Option B - Save Path DF To Its Own CSV

In [ ]:
# import os
# os.makedirs(out_dir, exist_ok=True)
# path_df.to_csv(os.path.join(out_dir, filename), index=False)

Hope this was helpful

--Calvin